In [46]:
import os
import random

from cryptography.hazmat.primitives import hashes

In [16]:
'''
q = 8380417
d = 13
tau = 39
challenge netropy = 192
gamma1 = 2^17
gamma2 = 95232
(k, l) = (4, 4)
eta = 2
beta = 78
omega = 80
reps = 4.25
'''

'\nq = 8380417\nd = 13\nGAMMA = 39\nchallenge netropy = 192\ngamma1 = 2^17\ngamma2 = 95232\n(k, l) = (4, 4)\neta = 2\nbeta = 78\nomega = 80\nreps = 4.25\n'

In [8]:
def Gen():
    #01
    zeta = os.urandom(32)
    
    #02
    (rho, rhol, K) = H3(zeta)
    
    #03
    A = ExpandA(rho)
    Acirc = NTTmat(A)
    
    #04
    (s1, s2) = ExpandS(rhol)
    
    #05
    As1 = INTTvec(Acirc*NTTvec(s1))
    t = As1 + s2
    
    #06
    (t1, t0) = Power2Roundq(t,13)
    
    #07
    tr = H(rho + t1)
    
    #08 
    pk = (rho, t1)
    sk = (rho, K, tr, s1, s2, t0)
    return pk, sk   

In [2]:
def Sign(sk, M):
    sk = (rho, K, tr, s1, s2, t0)
    
    #09
    A = ExpandA(rho)
    Acirc = NTTmat(A)
    
    #10
    mu = H(tr + M)
    
    #11
    kappa = 0
    z = 0; h = 0
    
    #12 
    rhol = H(K + mu)
    
    #13
    s1circ = NTTvec(s1)
    s2circ = NTTvec(s2)
    t0circ = NTTvec(t0)
    while z == 0 & h == 0:
        
        #14
        y = ExpandMask(rhol, kappa)
        
        #15
        w = INTTvec(Acirc * NTTvec(y))
        
        #16
        w1 = HighBitsq(w, 2 * 95232)
        
        #17
        ctil = H(mu, w1)
        
        #18 
        c = SampleInBall(ctil)
        ccirc = NTT(c)
        
        #19
        cs1 = INTTvec(ccirc*s1circ)
        z = y + cs1
        
        #20
        cs2 = INTTvec(ccirc*s2circ)
        r0 = LowBitsq(w-cs2, 2 * 95232)
        
        #21 
        if size_of_elements_bige(z, 2^17 - 78) | size_of_elements_bige(e0, 95232 - 78):
            z = 0
            h = 0
        #22
        else:
            #23
            ct0 = INTTvec(ccirc * t0circ)
            h = MakeHintq(-ct0, w - cs2 + ct0, 2 * 95232)
            #24
            if size_of_elements_bige(ct0 , 95232) | number_of_1s_big(h, 80):
                z = 0
                h = 0
        #25
        kappa = kappa + l
    
    #26
    sigma = (ctil, z, h)
    return sigma

In [6]:
def Verify(pk, M, sigma):
    pk = (rho, t1)
    sigma = (ctil, z, h)
    #27    
    A = ExpandA(rho)
    Acirc = NTTmat(A)
    
    #28
    mu = H(H(rho + t1) + M)
    
    #29
    c = SampleInBall(ctil)
    
    #30
    wl1 = UseHintq(h, INTTvec((Acirc * NTTvec(z)) - (NTTvec(c) * NTTvec(t1*(2^13)))))
    
    #31
    return size_of_elements_low(z, 2^17 - 78) & ctil == H(mu + wl1) & number_of_1s_lowe(h, 80)

3


In [ ]:
pk, sk = Gen()

M = os.urandom(32)

sigma = Sign(sk, M)

done = Verify(pk, M, sigma)

print(done)

In [26]:
def H(x):
    digest = hashes.Hash(hashes.SHAKE256(int(32)))
    digest.update(x)
    return digest.finalize()

In [37]:
def H3(x):
    digest = hashes.Hash(hashes.SHAKE256(int(96)))
    digest.update(x)
    buffer = digest.finalize()
    return buffer[:32], buffer[32:64], buffer[-32:] 

In [ ]:
def ExpandA(x):
    return 0

In [ ]:
def ExpandS(x):
    return 0

In [ ]:
def ExpandMask(x1, x2):
    return 0

In [34]:
def NTTvec(a):
    psi = []
    
    t = 2^13
    
    m = 1
    
    while m < 2^13:
        t = t//2
        
        for i in range(m):
            j1 = 2 * i * t
            
            j2 = j1 + t - 1
            
            S = 'sadsadasdsadasda'
            
            for j in range(j1, j2+1):
                U = a[j]
                V = a[j + t] * S
                
                a[j] = (U + V) % q
                
                a[j + t] = (U - V) % q
                
        m = m*2
        
    return a

In [1]:
def INTTvec(a):
    return 0

In [ ]:
def NTTmat(M):
    return 0

In [ ]:
def INTTmat(M):
    return 0

In [55]:
def SampleInBall(rho):
    random.seed(a=rho, version=2)
    c = []
    for i in range(256):
        c.append(0)
        
    for i in range(256 - 39, 256):
        j = random.randint(0, i)
        s = random.randint(0, 1)
        c[i] = c[j]
        c[j] = (-1)^s
        
    return c

In [70]:
def Power2Roundq(r, d):
    r = r % 8380417
    
    r0 = r % (2^d)
    r0 = r0 - (2^(d-1))
    
    return ((r - r0)//(2^d)), r0

In [ ]:
def MakeHintq(z, r, alpha):
    r1 = HighBitsq(r, alpha)
    v1 = HighBitsq(r + z, alpha)
    
    return r1 != v1

In [ ]:
def UseHintq(h, r, alpha):
    m = (8380417 - 1)//alpha
    
    (r1, r0) = Decomposeq(r, alpha)
    
    if h == 1 & r0 > 0:
        return (r + 1) % m
    
    if h == 1 & r0 <= 0:
        return (r - 1) % m
    
    return r1

In [ ]:
def HighBitsq(r, alpha):
    (r1, r0) = Decomposeq(r, alpha)
    return r1

In [ ]:
def LowBitsq(r, alpha):
    (r1, r0) = Decomposeq(r, alpha)
    return r0

In [ ]:
def Decomposeq(r, alpha):
    r = r % 8380417
    
    r0 = r % alpha
    r0 = r0 - (alpha//2)
    
    if r - r0 == 8380417 - 1:
        r1 = 0
        r0 = r0 - 1
    else:
        r1 = (r - r0)//alpha
        
    return (r1, r0)

In [89]:
def size_of_elements_low(w, cond):
    x = w % 8380417
    x = x - (8380417//2)
    if x < 0:
        x = -x
        
    return x < cond

In [90]:
def size_of_elements_bige(w, cond):
    x = w % 8380417
    x = x - (8380417//2)
    if x < 0:
        x = -x
        
    return x >= cond

In [91]:
def number_of_1s_big(w, cond):
    counter = 0
    aux = w
    while aux > 0:
        if aux % 2:
            counter += 1
        aux = aux // 2
        
    return counter > cond

In [92]:
def number_of_1s_lowe(w, cond):
    counter = 0
    aux = w
    while aux > 0:
        if aux % 2:
            counter += 1
        aux = aux // 2
        
    return counter <= cond